![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/0bd67cf0-43d5-46d2-879c-a752cae4c8e3)

# Install Necessary Libraries

In [1]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Success

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Clone a GitHub Repo locally

In [3]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = repo_url.split("/")[-1]  # Extract repository name from URL
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return str(repo_path)

In [4]:
path = [clone_repository("https://github.com/CoderAgent/SecureAgent"),clone_repository("https://github.com/Ashan-264/ChurnPredictor")]
path_source= ["https://github.com/CoderAgent/SecureAgent","https://github.com/Ashan-264/ChurnPredictor"]

In [5]:
print(path)
print(path_source)

['/content/SecureAgent', '/content/ChurnPredictor']
['https://github.com/CoderAgent/SecureAgent', 'https://github.com/Ashan-264/ChurnPredictor']


In [6]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [7]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_paths: list[str]):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    repo_files = {}


    try:
        for repo_path in repo_paths:
          files_content = []
          try:
            for root, _, files in os.walk(repo_path):
                # Skip if current directory is in ignored directories
                if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                    continue

                # Process each file in current directory
                for file in files:
                    file_path = os.path.join(root, file)
                    if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                        file_content = get_file_content(file_path, repo_path)
                        if file_content:
                            files_content.append(file_content)
            repo_files[repo_path] = files_content
          except Exception as e:
            print(f"Error reading repository: {str(e)}")
    except Exception as e:
        print(f"Error reading list of repositories: {str(e)}")

    return repo_files

In [8]:
repository_files = get_main_files_content(path)

In [9]:
repository_files

{'/content/SecureAgent': [{'name': 'src/env.ts',
   'content': 'import * as dotenv from "dotenv";\nimport { createPrivateKey } from "crypto";\nimport chalk from "chalk";\n\ndotenv.config();\n\nexport const env = {\n  GITHUB_APP_ID: process.env.GITHUB_APP_ID,\n  GITHUB_PRIVATE_KEY: process.env.GITHUB_PRIVATE_KEY,\n  GITHUB_WEBHOOK_SECRET: process.env.GITHUB_WEBHOOK_SECRET,\n  GROQ_API_KEY: process.env.GROQ_API_KEY,\n} as const;\n\nlet valid = true;\n\nfor (const key in env) {\n  if (!env[key as keyof typeof env]) {\n    console.log(\n      chalk.red("✖") +\n        chalk.gray(" Missing required env var: ") +\n        chalk.bold(`process.env.${key}`)\n    );\n    valid = false;\n  }\n}\n\ntry {\n  createPrivateKey(env.GITHUB_PRIVATE_KEY);\n} catch (error) {\n  console.log(\n    chalk.red(\n      "\\n✖ Invalid GitHub private key format for " +\n        chalk.bold(`process.env.GITHUB_PRIVATE_KEY`) +\n        "\\n"\n    ) +\n      chalk.gray("  • Must start with: ") +\n      chalk.bold("---

# Embeddings

In [10]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [11]:
text = "I am a programmer"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
embeddings

array([ 1.81737728e-02, -3.02659534e-03, -4.77465056e-02,  1.86379589e-02,
        3.14538032e-02,  1.87255573e-02, -1.52534526e-02, -6.77293763e-02,
       -1.26904100e-02,  1.28427539e-02,  5.80701083e-02,  4.00234982e-02,
        3.27073336e-02,  7.12998286e-02,  5.56373261e-02,  1.68628842e-02,
        6.97604120e-02, -5.02619967e-02,  6.13144785e-03, -1.46559263e-02,
       -4.51954128e-03,  4.82934080e-02, -2.53051482e-02, -1.97865185e-03,
       -4.36902903e-02, -2.41507702e-02,  1.29505172e-02, -3.78603698e-03,
       -2.05719229e-02,  1.09819129e-01,  3.07674590e-03, -2.80443188e-02,
       -1.55807380e-02, -1.24790529e-02,  1.75239245e-06, -2.93755950e-03,
       -1.43048624e-02,  4.88386266e-02, -6.21115230e-02,  2.95061693e-02,
       -1.40470909e-02,  2.20707953e-02,  1.13067841e-02,  4.70892675e-02,
        7.58306822e-03, -8.30540375e-05,  6.67821839e-02, -1.21320244e-02,
        4.39395290e-03,  2.47454122e-02,  1.02529097e-02, -6.54437998e-03,
       -5.53141721e-03, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [13]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [14]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-14-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-14-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [15]:
# Define a function to split text into smaller chunks
def split_text_into_chunks(text, chunk_size=1000, overlap=200):
    """
    Splits text into chunks of size `chunk_size` with `overlap` between chunks.
    """
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

for gitHubURL in path: #change to include url in one and path in another for key-value pair
  documents = []
  for file in repository_files[gitHubURL]:
      file_content = file['content']
      file_name = file['name']

      # Split the file content into smaller chunks
      content_chunks = split_text_into_chunks(file_content, chunk_size=4096)  # Adjust size as needed

      # Create a Document for each chunk
      for chunk_index, chunk in enumerate(content_chunks):
          doc = Document(
              page_content=chunk,
              metadata={
                  "source": file_name,
                  "chunk_index": chunk_index,
                  "total_chunks": len(content_chunks)
              }
          )
          documents.append(doc)


  vectorstore = PineconeVectorStore.from_documents(
      documents=documents,
      embedding=HuggingFaceEmbeddings(),
      index_name="codebase-rag",
      namespace= gitHubURL
  )

<ipython-input-15-50d8bfa5dc73>:35: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your Groq API Key [here](https://console.groq.com/keys)

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

![Screenshot 2024-11-25 at 12 00 16 AM](https://github.com/user-attachments/assets/e5525d29-bca6-4dbd-892b-cc770a6b281d)


In [16]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [60]:
query = "what does the code in each repo do?"
repositories = list(repository_files.keys())
print(repositories)

['/content/SecureAgent', '/content/ChurnPredictor']


In [61]:
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

array([ 4.00477722e-02, -1.28499977e-02, -3.38756219e-02,  5.19898674e-03,
       -5.70936501e-02,  4.15090173e-02,  6.55847555e-03,  2.33815778e-02,
       -2.54917350e-02,  1.46313086e-02, -8.56364798e-03,  3.10760401e-02,
        8.62860829e-02,  2.81283744e-02,  2.23100223e-02, -5.23197372e-03,
       -1.78713854e-02, -2.11915877e-02, -2.58849445e-03, -3.08912136e-02,
       -1.78432252e-04,  5.64120419e-04,  4.31205286e-03,  3.28482352e-02,
        5.06209321e-02, -4.14179265e-03,  6.07824838e-03,  6.04524091e-02,
       -2.53792163e-02,  2.27725934e-02,  2.47406587e-02,  4.49823253e-02,
        1.86768360e-02,  2.32302640e-02,  1.37002974e-06,  1.58906952e-02,
        1.27326259e-02,  2.38998663e-02,  1.90664083e-02,  3.68290581e-02,
       -2.73756087e-02, -1.85795072e-02,  6.29430488e-02,  1.10557452e-02,
        6.04899116e-02,  2.35471241e-02,  3.75435338e-03, -6.62549818e-03,
        1.30468337e-02,  6.47053728e-03,  2.10291836e-02,  1.54402591e-02,
       -3.30224149e-02,  

In [62]:
# Feel free to change the "top_k" parameter to be a higher or lower number
top_matches = {}
for repo in repositories:
  top_matches[repo] = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace=gitHubURL)



In [63]:
top_matches

{'/content/SecureAgent': {'matches': [{'id': '9a6e0f6e-a441-42e6-b526-65a7ab2fa311',
               'metadata': {'chunk_index': 9.0,
                            'source': 'Colab Files for Bank and Gym '
                                      'data/GymChurnPredictor.ipynb',
                            'text': '0   \\n",\n'
                                    '              "50%           1.000000      '
                                    '0.000000    29.000000   \\n",\n'
                                    '              "75%           6.000000      '
                                    '1.000000    31.000000   \\n",\n'
                                    '              "max          12.000000      '
                                    '1.000000    41.000000   \\n",\n'
                                    '              "\\n",\n'
                                    '              "       '
                                    'Avg_additional_charges_total  '
                              

In [64]:
contexts = {}
for repo_key in top_matches.keys():
  contexts[repo_key] = [item['metadata']['text'] for item in top_matches[repo_key]['matches']]

In [65]:
contexts

{'/content/SecureAgent': ['0   \\n",\n              "50%           1.000000      0.000000    29.000000   \\n",\n              "75%           6.000000      1.000000    31.000000   \\n",\n              "max          12.000000      1.000000    41.000000   \\n",\n              "\\n",\n              "       Avg_additional_charges_total  Month_to_end_contract     Lifetime  \\\\\\n",\n              "count                   4000.000000            4000.000000  4000.000000   \\n",\n              "mean                     146.943728               4.322750     3.724750   \\n",\n              "std                       96.355602               4.191297     3.749267   \\n",\n              "min                        0.148205               1.000000     0.000000   \\n",\n              "25%                       68.868830               1.000000     1.000000   \\n",\n              "50%                      136.220159               1.000000     3.000000   \\n",\n              "75%                      210

In [93]:
augmented_query = ""
for repo_key in contexts.keys():
  augmented_query += f"<CONTEXT>{repo_key}\n" + "\n\n-------\n\n".join(contexts[repo_key][ : 2]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [94]:
print(augmented_query)

<CONTEXT>/content/SecureAgent
0   \n",
              "50%           1.000000      0.000000    29.000000   \n",
              "75%           6.000000      1.000000    31.000000   \n",
              "max          12.000000      1.000000    41.000000   \n",
              "\n",
              "       Avg_additional_charges_total  Month_to_end_contract     Lifetime  \\\n",
              "count                   4000.000000            4000.000000  4000.000000   \n",
              "mean                     146.943728               4.322750     3.724750   \n",
              "std                       96.355602               4.191297     3.749267   \n",
              "min                        0.148205               1.000000     0.000000   \n",
              "25%                       68.868830               1.000000     1.000000   \n",
              "50%                      136.220159               1.000000     3.000000   \n",
              "75%                      210.949625               6

In [96]:
system_prompt = f"""You are a Senior Software Engineer

Provide answers to questions based on the code provided. Always consider all of the context given in the <Context> tags when forming a response. Present your answers in a concise, structured, and user-friendly format.

**Instructions for Formatting**:
- Split the answers clearly based on the repositories mentioned in the <Context>/content<repository name> tags.
- Use bullet points or numbered lists for clarity where applicable.
- Provide summaries in plain, easy-to-read language.
- If there are multiple repositories, ensure each one has its own clear section header with a brief explanation of its purpose.
- Avoid technical jargon unless necessary, and provide examples where helpful.
- Ensure unrelated repositories or snippets are separated under an "Other Repositories" section.

**Response Format Example**:
**<Repository Name>**
- Purpose: Briefly explain what the repository is for.
- Key Features:
  - Highlight specific functionalities or code blocks in simple terms.
  - Provide examples or notable snippets.
- Overall Summary: Conclude with a concise description of what the repository focuses on.

**Other Repositories**
- Briefly summarize other repositories if context is unclear or limited."""

# LLM call
llm_response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

# Extract raw response
response = llm_response.choices[0].message.content



In [97]:
response

'Based on the provided code, there are two repositories: SecureAgent and ChurnPredictor.\n\n**SecureAgent**\n\nThe SecureAgent repository appears to be a Python script that makes a request to an API endpoint and prints the response. The code checks if the response status code is 200 (OK), and if so, it parses the JSON response and prints it. If the response status code is not 200, it prints an error message with the status code and response text.\n\nThis script does not seem to involve any specific functionality related to SecureAgent.\n\n**ChurnPredictor**\n\nThe ChurnPredictor repository appears to be a collection of data and statistics about churn prediction. The code provides the following information:\n\n* Counts, means, standard deviations, and percentiles for several columns, including:\n\t+ Avg_additional_charges_total\n\t+ Month_to_end_contract\n\t+ Lifetime\n\t+ Avg_class_frequency_total\n\t+ Avg_class_frequency_current_month\n\t+ Churn\n\nThe code also includes a table with 

In [98]:
def format_response(raw_response):
    # Split the response into sections based on repository headings
    sections = raw_response.split("\n\n")
    formatted_response = []

    for section in sections:
        if section.startswith("**"):
            # Extract repository name and content
            repo_name = section.split("\n")[0].replace("**", "").strip()
            repo_content = "\n".join(section.split("\n")[1:]).strip()
            formatted_response.append(f"### {repo_name}\n{repo_content}")
        else:
            # Add any unstructured text or comments
            formatted_response.append(f"{section.strip()}")

    return "\n\n".join(formatted_response)

formatted_response = format_response(response)
print(formatted_response)

Based on the provided code, there are two repositories: SecureAgent and ChurnPredictor.

### SecureAgent


The SecureAgent repository appears to be a Python script that makes a request to an API endpoint and prints the response. The code checks if the response status code is 200 (OK), and if so, it parses the JSON response and prints it. If the response status code is not 200, it prints an error message with the status code and response text.

This script does not seem to involve any specific functionality related to SecureAgent.

### ChurnPredictor


The ChurnPredictor repository appears to be a collection of data and statistics about churn prediction. The code provides the following information:

* Counts, means, standard deviations, and percentiles for several columns, including:
	+ Avg_additional_charges_total
	+ Month_to_end_contract
	+ Lifetime
	+ Avg_class_frequency_total
	+ Avg_class_frequency_current_month
	+ Churn

The code also includes a table with multiple rows and columns

# Putting it all together

In [70]:
# def perform_rag(query):
#     raw_query_embedding = get_huggingface_embeddings(query)

#     top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace=gitHubURL)

#     # Get the list of retrieved texts
#     contexts = [item['metadata']['text'] for item in top_matches['matches']]

#     augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

#     # Modify the prompt below as need to improve the response quality
#     system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

#     Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
#     """

#     llm_response = client.chat.completions.create(
#         model="llama-3.1-8b-instant",
#         messages=[
#             {"role": "system", "content": system_prompt},
#             {"role": "user", "content": augmented_query}
#         ]
#     )

#     return llm_response.choices[0].message.content

In [71]:
import time
from collections import defaultdict

# Define fallback models
FALLBACK_MODELS = {
    "mixtral-8x7b-32768": ["llama-3.1-8b-instant","gemma-7b-it", "gemma2-9b-it", "llama-3.1-70b-versatile", "llama-3.2-11b-text-preview", "llama-3.2-11b-vision-preview", "llama-3.2-1b-preview", "llama-3.2-3b-preview", "llama-3.2-90b-text-preview", "llama-3.2-90b-vision-preview", "llama-guard-3-8b", "llama3-70b-8192", "llama3-8b-8192", "llama3-groq-70b-8192-tool-use-preview", "llama3-groq-8b-8192-tool-use-preview", "llava-v1.5-7b-4096-preview", "mixtral-8x7b-32768"
],
    "llama-3.1-70b-versatile": ["llama-3.2-1b-preview", "llama-3.2-3b-preview"],
}

def perform_rag(query, primary_model="mixtral-8x7b-32768"):
    """
    Perform RAG (retrieval-augmented generation) with fallback models
    when the primary model fails due to rate limits or errors.
    """
    fallback_models = FALLBACK_MODELS.get(primary_model, [])
    models_to_try = [primary_model] + fallback_models  # Primary + fallback models

    for model in models_to_try:
        try:
            # Perform the RAG operation
            raw_query_embedding = get_huggingface_embeddings(query)

            top_matches = pinecone_index.query(
                vector=raw_query_embedding.tolist(),
                top_k=5,
                include_metadata=True,
                namespace=gitHubURL
            )

            # Get the list of retrieved texts
            contexts = [item['metadata']['text'] for item in top_matches['matches']]

            augmented_query = (
                "<CONTEXT>\n" +
                "\n\n-------\n\n".join(contexts[:10]) +
                "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" +
                query
            )

            # Modify the prompt below as needed to improve the response quality
            system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

            Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
            """



            llm_response = client.chat.completions.create(
                model=model,
                messages=[
                    #{"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"{augmented_query}  {system_prompt}"}
                ]
            )

            print(f"Response successfully retrieved from {model}")
            return llm_response.choices[0].message.content

        except Exception as e:
            error_message = str(e)  # Capture the exception as a string

            if "max_tokens" in error_message.lower():
              print("Maximum token limit reached.")
            elif "rate_limit" in error_message.lower():
              print("Rate limit exceeded.")

            print(f"Error occurred with model {model} \n {error_message}")

            print(f"Trying next fallback model...")

    raise Exception("All models failed due to rate limits or other errors.")

# Example usage
try:
    response = perform_rag("Explain how to integrate OAuth in React.")
    print(response)
except Exception as e:
    print(f"Failed to get a response: {e}")


Rate limit exceeded.
Error occurred with model mixtral-8x7b-32768 
Error code: 413 - {'error': {'message': 'Request too large for model `mixtral-8x7b-32768` in organization `org_01j86d1a6cf4nb7efk0wy8bqfv` on tokens per minute (TPM): Limit 5000, Requested 15359, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Trying next fallback model...
Response successfully retrieved from llama-3.1-8b-instant
I'd be happy to help you integrate OAuth in React.

Integrating OAuth into a React application involves several steps:

1.  **Choose an OAuth provider**: You'll need to decide which OAuth provider you want to use for authentication. Some popular options include Google, Facebook, GitHub, and Auth0.
2.  **Register your application**: You'll need to register your application with the OAuth provider. This will give you a client ID and client secret that you'll use to authenticate

In [72]:
for i in range(1):
  response = perform_rag("what are the coding files in the repo?")

  print(response)

Rate limit exceeded.
Error occurred with model mixtral-8x7b-32768 
Error code: 413 - {'error': {'message': 'Request too large for model `mixtral-8x7b-32768` in organization `org_01j86d1a6cf4nb7efk0wy8bqfv` on tokens per minute (TPM): Limit 5000, Requested 18255, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Trying next fallback model...
Response successfully retrieved from llama-3.1-8b-instant
I see that this prompt has nothing to do with the JSON object and the Python code you provided at the beginning of our conversation. It seems to be a creative writing prompt for a story that involves a multiverse, dimensions, and a protagonist on a quest.

If you would like me to help you with the Python code and the JSON object instead, I'd be more than happy to assist you. However, if you'd like to proceed with this prompt, I can certainly provide guidance and offer assista

In [73]:
model_list = ["gemma-7b-it", "gemma2-9b-it", "llama-3.1-70b-versatile", "llama-3.1-8b-instant", "llama-3.2-11b-text-preview", "llama-3.2-11b-vision-preview", "llama-3.2-1b-preview", "llama-3.2-3b-preview", "llama-3.2-90b-text-preview", "llama-3.2-90b-vision-preview", "llama-guard-3-8b", "llama3-70b-8192", "llama3-8b-8192", "llama3-groq-70b-8192-tool-use-preview", "llama3-groq-8b-8192-tool-use-preview", "llava-v1.5-7b-4096-preview", "mixtral-8x7b-32768"
]

NGROK URL WEB link

In [74]:
!pip install pyngrok


In [75]:
from pyngrok import ngrok

# Replace "your_auth_token" with your ngrok authentication token
ngrok_token = userdata.get("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(ngrok_token)


In [76]:
from http.server import SimpleHTTPRequestHandler
import socketserver
import threading

PORT = 5000

def start_server():
    handler = SimpleHTTPRequestHandler
    with socketserver.TCPServer(("", PORT), handler) as httpd:
        print("Serving at port", PORT)
        httpd.serve_forever()

# Run the server in a separate thread
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()


Exception in thread 

In [77]:
# Create an ngrok tunnel for port 5000
public_url = ngrok.connect(5000)
print("Public URL:", public_url)


Thread-30 (start_server):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-76-e942150da11e>", line 9, in start_server
  File "/usr/lib/python3.10/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use


Public URL: NgrokTunnel: "https://8388-34-142-145-119.ngrok-free.app" -> "http://localhost:5000"
